<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hu_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-obnuocs3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-obnuocs3
  Resolved https://github.com/huggingface/transformers to commit 25b0f2033ba23e354ef2f665764248fcbb3f49ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8001073 sha256=37be12be58f73d4f8a347a6c6a7963dab1f6f41b43dbd8334475458b84fc735c
  Stored in directory: /tmp/pip-ephem-wheel-cache-28zme65e/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hu", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("tiny")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=8aaebf042943b20e82787da3e7d3481e16ecf03d986576d17084d0c20975ddbe
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [7]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 91.7MiB/s]


In [8]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [9]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hu")
    return result["text"]

In [10]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [11]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [12]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


 # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.7333333333333333, Precision: 0.35294117647058826, Recall: 0.4, F1: 0.37500000000000006
Processed 2 files, WER: 0.2727272727272727, Precision: 0.7272727272727273, Recall: 0.8, F1: 0.761904761904762
Processed 3 files, WER: 0.5263157894736842, Precision: 0.4411764705882353, Recall: 0.45454545454545453, F1: 0.4477611940298507
Processed 4 files, WER: 0.391304347826087, Precision: 0.6190476190476191, Recall: 0.5909090909090909, F1: 0.6046511627906977
Processed 5 files, WER: 0.35294117647058826, Precision: 0.6875, Recall: 0.6470588235294118, F1: 0.6666666666666667
Processed 6 files, WER: 0.4722222222222222, Precision: 0.5483870967741935, Recall: 0.53125, F1: 0.5396825396825397
Processed 7 files, WER: 0.5, Precision: 0.5714285714285714, Recall: 0.5, F1: 0.5333333333333333
Processed 8 files, WER: 0.38235294117647056, Precision: 0.6551724137931034, Recall: 0.7037037037037037, F1: 0.6785714285714286
Processed 9 files, WER: 1.0, Precision: 0.047619047619047616, Recall: 0.